In [1]:
import os
import itertools
import math
import numpy as np
import pandas as pd
import requests
import pickle
from datetime import datetime as dt
from datetime import timedelta
import pytz
from pytz import timezone
from pprint import pprint
import pandas_datareader.data as web
from collections import namedtuple
from operator import itemgetter

import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import yfinance as yf
import fundamentalanalysis as fa
import streamlit as st

import holidays
import pandas_datareader.data as web
import yahoo_fin.stock_info as si
from urllib.request import Request, urlopen
from html_table_parser.parser import HTMLTableParser 
import streamlit as st

from load_data import *
from get_data import get_tickers

pd.options.plotting.backend = "plotly"
# pd.set_option('display.max_rows', None)


In [19]:
us_holidays = holidays.US(2012)
us_holidays

{datetime.date(2012, 1, 1): "New Year's Day", datetime.date(2012, 1, 2): "New Year's Day (Observed)", datetime.date(2012, 1, 16): 'Martin Luther King Jr. Day', datetime.date(2012, 2, 20): "Washington's Birthday", datetime.date(2012, 5, 28): 'Memorial Day', datetime.date(2012, 7, 4): 'Independence Day', datetime.date(2012, 9, 3): 'Labor Day', datetime.date(2012, 10, 8): 'Columbus Day', datetime.date(2012, 11, 11): 'Veterans Day', datetime.date(2012, 11, 12): 'Veterans Day (Observed)', datetime.date(2012, 11, 22): 'Thanksgiving', datetime.date(2012, 12, 25): 'Christmas Day'}

In [14]:
df = get_ticker_data('a')
df.index = pd.to_datetime(df.index, utc=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5844 entries, 1999-11-18 05:00:00+00:00 to 2023-02-09 05:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5844 non-null   float64
 1   High       5844 non-null   float64
 2   Low        5844 non-null   float64
 3   Close      5844 non-null   float64
 4   Adj Close  5844 non-null   float64
 5   Volume     5844 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 319.6 KB


In [11]:
tz = timezone('EST')

df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5844 entries, 1999-11-18 05:00:00+00:00 to 2023-02-09 05:00:00+00:00
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       5844 non-null   float64
 1   High       5844 non-null   float64
 2   Low        5844 non-null   float64
 3   Close      5844 non-null   float64
 4   Adj Close  5844 non-null   float64
 5   Volume     5844 non-null   int64  
dtypes: float64(5), int64(1)
memory usage: 319.6 KB


In [10]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-11-18 05:00:00+00:00,32.546494,35.765381,28.612303,31.473534,26.889660,62546380
1999-11-19 05:00:00+00:00,30.713518,30.758226,28.478184,28.880545,24.674316,15234146
1999-11-22 05:00:00+00:00,29.551144,31.473534,28.657009,31.473534,26.889660,6577870
1999-11-23 05:00:00+00:00,30.400572,31.205294,28.612303,28.612303,24.445148,5975611
1999-11-24 05:00:00+00:00,28.701717,29.998213,28.612303,29.372318,25.094477,4843231


In [5]:
# Select country
us_holidays = holidays.US([2020, 2021, 2022, 2023])

# If it is a holidays then it returns True else False
# print('01-01-2018' in us_holidays)
# print('02-01-2018' in us_holidays)

# # What holidays is it?
# print(us_holidays.get('01-01-2018'))
# print(us_holidays.get('02-01-2018'))
pprint(us_holidays)

{datetime.date(2020, 1, 1): "New Year's Day",
 datetime.date(2020, 1, 20): 'Martin Luther King Jr. Day',
 datetime.date(2020, 2, 17): "Washington's Birthday",
 datetime.date(2020, 5, 25): 'Memorial Day',
 datetime.date(2020, 7, 3): 'Independence Day (Observed)',
 datetime.date(2020, 7, 4): 'Independence Day',
 datetime.date(2020, 9, 7): 'Labor Day',
 datetime.date(2020, 10, 12): 'Columbus Day',
 datetime.date(2020, 11, 11): 'Veterans Day',
 datetime.date(2020, 11, 26): 'Thanksgiving',
 datetime.date(2020, 12, 25): 'Christmas Day',
 datetime.date(2021, 1, 1): "New Year's Day",
 datetime.date(2021, 1, 18): 'Martin Luther King Jr. Day',
 datetime.date(2021, 2, 15): "Washington's Birthday",
 datetime.date(2021, 5, 31): 'Memorial Day',
 datetime.date(2021, 6, 18): 'Juneteenth National Independence Day (Observed)',
 datetime.date(2021, 6, 19): 'Juneteenth National Independence Day',
 datetime.date(2021, 7, 4): 'Independence Day',
 datetime.date(2021, 7, 5): 'Independence Day (Observed)',
 da

In [4]:
rf_rates = get_rf_data()
SPY_df = get_SPY_data()
SPY_info_df = get_SPY_info()
ticker_info = get_ticker_info()
ticker_list = SPY_info_df.index.to_list()
sector_list = SPY_info_df['Sector'].unique()
first_date = SPY_df.iloc[0].name
last_date = SPY_df.iloc[-1].name
yr_ago = last_date - timedelta(days=365)
# TTM_ratios, ratios_data_report = load_TTM_ratios()

In [4]:
def get_removed_market_data():  
    '''Get historical data for tickers removed from S&P 500'''

    tickers = get_tickers()[1]
    n = len(tickers)
    not_downloaded = []
    path = r'data\removed_from_index\market_data'

    for i, ticker in enumerate(tickers, 1):
        try:
            fname = os.path.join(path, f'{ticker}.csv')
            df = si.get_data(ticker) # download stock data
            df.to_csv(fname)
            print(f"\r{i}/{n} ({i / n:.2%}) of removed SPY market data downloaded",
                  end='', flush=True)
        except Exception as e:
            # print(f'\r{i}/{n}: {ticker} - {e}')
            not_downloaded.append(ticker)  

    print('\nS&P 500 removed stock data downloaded \n')

    if not_downloaded:
        print(f'{len(not_downloaded)} stocks not downloaded \n')
    
    return not_downloaded

In [3]:
tickers, hist_tickers = get_tickers()
# hist_tickers = set(hist_tickers) - set(tickers)
len(hist_tickers)

306

In [4]:
path = r'data\removed_from_index\market_data'
hist_tickers = [os.path.splitext(f)[0] for f in os.listdir(path)]
print(len(hist_tickers))
# hist_tickers

185


In [14]:
ticker_info['ABMD']

{'Security': 'N/A',
 'Sector': 'N/A',
 'Industry': 'N/A',
 'Business Summary': 'N/A',
 'Website': 'N/A'}

In [107]:
securities_list = fa.available_companies('a812dc7d35321ed0502ea50d8dc71be9')

In [108]:
len(securities_list)

67339

In [5]:
securities_list['type'].unique()

array(['etf', 'stock', 'trust', 'fund'], dtype=object)

In [6]:
securities_list[securities_list['type'] == 'etf']

,name,price,exchange,exchangeShortName,type
symbol,,,,,
SPY,SPDR S&P 500 ETF Trust,396.91,New York Stock Exchange Arca,AMEX,etf
GDX,VanEck Gold Miners ETF,24.19,New York Stock Exchange Arca,AMEX,etf
EEM,iShares MSCI Emerging Markets ETF,38.25,New York Stock Exchange Arca,AMEX,etf
XLF,Financial Select Sector SPDR Fund,33.74,New York Stock Exchange Arca,AMEX,etf
EFA,iShares MSCI EAFE ETF,60.49,New York Stock Exchange Arca,AMEX,etf
...,...,...,...,...,...
ASOL.SW,21Shares Solana staking ETP,19.08,Swiss Exchange,SIX,etf
AVAX.SW,21Shares Avalanche ETP,3.63,Swiss Exchange,SIX,etf
ABNB.SW,21Shares Binance BNB ETP,20.12,Swiss Exchange,SIX,etf


## Technical Analysis

- S/R Levels
- Volume Analysis
- Peak-Troughs Analysis

## Peak-Troughs Analysis

In [2]:
from scipy.signal import find_peaks, peak_prominences
from sklearn import preprocessing
from scipy.signal import argrelextrema

pd.options.display.float_format = "{:,.2f}".format

In [3]:
end = dt.now()
years = 1
days = 180
# days = days + 3 if end.weekday() == 0 else days
end = end - timedelta(3) if end.weekday() == 0 else end
start = end - timedelta(days * years)

# start = end - timedelta(20)
# print(divmod((end - start).days, 365))
print(f'start: {start.date()}, end: {end.date()}')

start: 2022-08-18, end: 2023-02-14


In [14]:
ticker = 'aap'
df = get_ticker_data(ticker)[start:end]
df_5m = get_interval_market_data(ticker, '5 Min')
df = df_5m.copy()[end - timedelta(5):end]
open = df['Open'] 
high = df['High']
low = df['Low']
close = df['Close']
volume = df['Volume']

C:\Users\pnjor\AppData\Local\Temp\ipykernel_3036\1486397088.py:4: FutureWarning:

Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version. Use a timezone-aware object instead.



In [19]:
p = find_peaks(close)[0]
prominences = peak_prominences(close, p)[0]
height = close[p].to_numpy() - prominences

for i in range(len(p)):
    print(f'{close[p[i]]:.2f} - {prominences[i]:.2f} = {height[i]:.2f}')

152.01 - 0.10 = 151.92
152.73 - 0.35 = 152.38
153.50 - 0.46 = 153.04
153.90 - 0.58 = 153.32
153.86 - 0.11 = 153.75
154.04 - 0.15 = 153.89
154.05 - 0.16 = 153.89
154.22 - 0.13 = 154.09
154.59 - 0.32 = 154.26
154.68 - 0.56 = 154.12
154.85 - 3.30 = 151.55
154.49 - 0.04 = 154.45
153.45 - 0.41 = 153.04
153.12 - 0.04 = 153.08
152.45 - 0.16 = 152.29
152.58 - 0.38 = 152.21
151.40 - 0.03 = 151.37
151.92 - 0.70 = 151.22
149.86 - 0.09 = 149.77
151.19 - 0.05 = 151.14
151.71 - 1.26 = 150.46
151.48 - 0.56 = 150.92
151.27 - 0.33 = 150.94
151.30 - 0.15 = 151.15
151.40 - 0.26 = 151.14
151.29 - 0.01 = 151.28
151.43 - 0.70 = 150.73
151.03 - 0.24 = 150.79
150.90 - 0.06 = 150.84
151.60 - 1.03 = 150.57
151.32 - 0.08 = 151.24
150.90 - 0.01 = 150.90
151.18 - 0.13 = 151.04
151.44 - 0.01 = 151.43
152.20 - 0.01 = 152.19


In [20]:
peaks = argrelextrema(close.to_numpy(), np.greater)[0]
troughs = argrelextrema(close.to_numpy(), np.less)[0]
print(f'peaks: {len(peaks)}, troughs: {len(troughs)}')
df_peaks = pd.DataFrame({'peaks': close[peaks]})
df_troughs = pd.DataFrame({'troughs': close[troughs]})
# df_peaks_troughs = pd.concat([df_peaks, df_troughs], axis=0)
close1 = min(peaks, troughs, key=itemgetter(0))
close2 = max(peaks, troughs, key=itemgetter(0))
dy0 = abs(close[close1[0]] - close[0])
dist = [dy0]
peak_troughs = []

print(f'{close[close1[0]]:,.2f} - {close[0]:,.2f} = {dy0:,.2f}')

for i, (y1, y2) in enumerate(zip(close1, close2)):
    dy = abs(close[y2] - close[y1])
    dist.append(dy)
    # print(dist)
    if i == 0:
        retracement = dy / dy0
    else:
        retracement = dist[i] / dist[i - 1]
        
    print(f'{i}: {close[y2]:,.2f} - {close[y1]:,.2f} = {dy:,.2f} | {dist[i]:,.2f} / {dist[i - 1]:,.2f} = ({retracement:.2})')

    if retracement >= 0.32:
        peak_troughs.extend([y1, y2])
    
    

peak_troughs = list(set(peak_troughs))
# peak_troughs

peaks: 35, troughs: 34
152.01 - 151.55 = 0.46
0: 151.92 - 152.01 = 0.10 | 0.46 / 0.10 = (0.21)
1: 152.38 - 152.73 = 0.35 | 0.10 / 0.46 = (0.21)
2: 153.04 - 153.50 = 0.46 | 0.35 / 0.10 = (3.7)
3: 153.32 - 153.90 = 0.58 | 0.46 / 0.35 = (1.3)
4: 153.75 - 153.86 = 0.11 | 0.58 / 0.46 = (1.3)
5: 153.89 - 154.04 = 0.15 | 0.11 / 0.58 = (0.19)
6: 153.89 - 154.05 = 0.16 | 0.15 / 0.11 = (1.4)
7: 154.09 - 154.22 = 0.13 | 0.16 / 0.15 = (1.1)
8: 154.26 - 154.59 = 0.32 | 0.13 / 0.16 = (0.82)
9: 154.12 - 154.68 = 0.56 | 0.32 / 0.13 = (2.4)
10: 154.45 - 154.85 = 0.40 | 0.56 / 0.32 = (1.7)
11: 153.04 - 154.49 = 1.46 | 0.40 / 0.56 = (0.71)
12: 153.08 - 153.45 = 0.37 | 1.46 / 0.40 = (3.6)
13: 152.29 - 153.12 = 0.84 | 0.37 / 1.46 = (0.25)
14: 152.21 - 152.45 = 0.24 | 0.84 / 0.37 = (2.3)
15: 151.37 - 152.58 = 1.21 | 0.24 / 0.84 = (0.29)
16: 151.22 - 151.40 = 0.18 | 1.21 / 0.24 = (5.1)
17: 149.09 - 151.92 = 2.83 | 0.18 / 1.21 = (0.15)
18: 149.77 - 149.86 = 0.09 | 2.83 / 0.18 = (1.6e+01)
19: 151.14 - 151.19 =

In [40]:
for dir_, folders, files in os.walk(r'C:\Users\pnjor\OneDrive\Documents\Books\Fiction\Raymond Feist Collections'):
    # for folder in folders:
    #     # print(folder)
    #     print(files)
        for file in files:
            print(file)
        

Raymond E. Feist.jpg
#1 Riftwar Saga #1 Magician.epub
#1 Riftwar Saga #2 Silverthorn.epub
#1 Riftwar Saga #3 Darkness at Sethanon.epub
Prince Arutha Image.jpg
#10 Chaoswar Saga #1 A Kingdom Besieged.epub
#10 Chaoswar Saga #2 A Crown Imperiled.epub
#10 Chaoswar Saga #3 Magician's End.epub
#2 Empire #1 Daughter of the Empire.epub
#2 Empire #2 Servant of the Empire.epub
#2 Empire #3 Mistress of the Empire.epub
#3 Krondor's Sons #1 Prince of the Blood.epub
#3 Krondor's Sons #2 King's Buccaneer.epub
#4 Serpentwar Saga #1 Shadow of a Dark Queen.epub
#4 Serpentwar Saga #2 Rise of a Merchant Prince.epub
#4 Serpentwar Saga #3 Rage of a Demon King.epub
#4 Serpentwar Saga #4 Shards of a Broken Crown.epub
#5 Riftwar Legacy #1 Krondor the Betrayal.epub
#5 Riftwar Legacy #2 Krondor the Assassins.epub
#5 Riftwar Legacy #3 Krondor Tear of the Gods.epub
#6 Legends of the Riftwar #1 Honoured Enemy.epub
#6 Legends of the Riftwar #2 Murder in LaMut.epub
#6 Legends of the Riftwar #3 Jimmy the Hand.epub
#7 

In [18]:
# peaks = height_ixs
# fig = go.Figure()
# fig.add_scatter(x=df.index, y=df['rtn'], name='Return', line_color='red', opacity=0.5)
# fig.add_scatter(x=df.index, y=df['log_rtn'], name='Log Return', line_color='blue', opacity=0.5)
# y = scaled_df['Close']
cs = go.Candlestick(x=df.index, 
                    open=df['Open'], 
                    high=df['High'],
                    low=df['Low'], 
                    close=df['Close'],
                    name=ticker)
cs.increasing.fillcolor = 'green'
cs.increasing.line.color = 'darkgreen'
cs.decreasing.fillcolor = 'red'
cs.decreasing.line.color = 'indianred'
fig = go.Figure()
    
fig.add_scatter(x=df.index, y=close, name='Close', line_color='blue', opacity=0.5)
fig.add_scatter(x=close[peaks].index,
                y=close[peaks],
                name='peaks',
                mode='markers',
                marker=dict(symbol='x', color='orange'), opacity=0.5)
fig.add_scatter(x=close[troughs].index,
                y=close[troughs],
                name='troughs',
                mode='markers',
                marker=dict(symbol='x', color='red'), opacity=0.5)
fig.add_scatter(x=close[peak_troughs].index,
                y=close[peak_troughs],
                name='peak_troughs',
                mode='markers',
                marker=dict(symbol='circle-open', color='green', size=8))
# fig.add_vline(x=close[peaks].index,
#                 # y=heights,
#                 # mode='lines',
#                 )
us_holidays = list(holidays.US(range(start.year, end.year + 1)).keys())
rangebreaks = [dict(bounds=["sat", "mon"]), dict(values=us_holidays)]
rangebreaks.extend([dict(bounds=[16, 9.5], pattern="hour")])
fig.update_xaxes(rangebreaks=rangebreaks)
fig.update_layout(hovermode="x unified", paper_bgcolor='gray', plot_bgcolor='gray')
fig

In [223]:
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
d = scaler.fit_transform(df)
scaled_df = pd.DataFrame(d, columns=df.columns)
scaled_df

,Open,High,Low,Close,Adj Close,Volume
0,0.29,0.29,0.33,0.33,0.32,0.19
1,0.28,0.26,0.31,0.30,0.30,0.34
2,0.72,0.69,0.54,0.55,0.55,1.00
3,0.53,0.49,0.50,0.49,0.48,0.32
4,0.46,0.43,0.46,0.43,0.42,0.24
...,...,...,...,...,...,...
120,0.82,0.84,0.85,0.82,0.82,0.04
121,0.79,0.85,0.84,0.88,0.88,0.14
122,0.85,0.86,0.90,0.88,0.88,0.12
123,0.90,0.89,0.86,0.82,0.82,0.06


In [29]:
1/6

0.16666666666666666

In [259]:
# df['rtn'] = df['Close'].pct_change()
# df['log_rtn'] = np.log1p(df['Close'].pct_change())
# df['Close'] = df['Volume'].pct_change()
peaks, _ = find_peaks(scaled_df['Close'])
promin = peak_prominences(scaled_df['Close'], peaks)[0]
heights = scaled_df['Close'][peaks] - promin
# peaks = peaks.tolist()
troughs, _ = find_peaks(scaled_df['Close'] * -1)
# troughs = troughs.tolist()
# peaks
# promin
height_ixs = []
for i, x in zip(heights.index, heights.values):
    if abs(x) >= 1/3:
        height_ixs.append(i)
        # print(x)
heights
# height_ixs


2     0.30
8     0.30
11    0.18
13    0.17
19    0.17
24    0.22
31    0.00
36    0.11
42    0.11
45    0.12
51    0.40
53    0.45
55    0.34
60    0.41
63    0.61
65    0.66
68    0.63
70    0.90
72    0.78
76    0.79
81    0.84
84    0.68
90    0.71
93    0.73
95    0.75
99    0.69
103   0.83
105   0.93
110   0.80
113   0.91
118   0.82
121   0.82
Name: Close, dtype: float64

In [163]:
x,y,z=[1,2,4]
x, y

(1, 2)

In [92]:
peaks, _ = find_peaks(df['Close'], height=0)
# troughs, _ = find_peaks(df['rtn'] * -1, height=0)
# peaks = df.iloc[peaks].index
# troughs = df.iloc[troughs].index
# waves = itertools.zip_longest(peaks, troughs)
# for i, (p, t) in enumerate(zip(peaks, troughs)):
#     p = 
#     if i > 0 and p[i-1] < p[i] and t[i-1] > t[i]:
#         print('Trending')
peaks
# troughs
# print(peaks[0], troughs[0])

array([], dtype=int64)

In [3]:
holidays.US(range(start.year, end.year + 1))


{datetime.date(2022, 1, 1): "New Year's Day", datetime.date(2022, 1, 17): 'Martin Luther King Jr. Day', datetime.date(2022, 2, 21): "Washington's Birthday", datetime.date(2022, 5, 30): 'Memorial Day', datetime.date(2022, 6, 19): 'Juneteenth National Independence Day', datetime.date(2022, 6, 20): 'Juneteenth National Independence Day (Observed)', datetime.date(2022, 7, 4): 'Independence Day', datetime.date(2022, 9, 5): 'Labor Day', datetime.date(2022, 10, 10): 'Columbus Day', datetime.date(2022, 11, 11): 'Veterans Day', datetime.date(2022, 11, 24): 'Thanksgiving', datetime.date(2022, 12, 25): 'Christmas Day', datetime.date(2022, 12, 26): 'Christmas Day (Observed)', datetime.date(2023, 1, 1): "New Year's Day", datetime.date(2023, 1, 2): "New Year's Day (Observed)", datetime.date(2023, 1, 16): 'Martin Luther King Jr. Day', datetime.date(2023, 2, 20): "Washington's Birthday", datetime.date(2023, 5, 29): 'Memorial Day', datetime.date(2023, 6, 19): 'Juneteenth National Independence Day', dat

## S/R Levels

In [10]:
def isSupport(df, i):
    '''Returns True if value is a price support level'''

    X = df['Low']
    support = X[i] < X[i - 1] \
                and X[i] < X[i + 1] \
                and X[i + 1] < X[i + 2] \
                and X[i - 1] < X[i - 2]

    return support


def isResistance(df, i):
    '''Returns True if value is a price resistance level'''

    X = df['High']
    resistance = X[i] > X[i - 1] \
                    and X[i] > X[i + 1] \
                    and X[i + 1] > X[i + 2] \
                    and X[i - 1] > X[i - 2] 

    return resistance
    

In [37]:
def sr_levels(df, start, end):
    '''Returns key support/S-R levels for a security'''

    df = df[start : end]
    df.columns = df.columns.str.title()
    df.drop(columns=['Ticker', 'Adjclose'], inplace=True)
    support = []
    resistance = []
    levels = []
    s = (df['High'] - df['Low']).mean()

    def isFarFromLevel(l):
        '''
        Given a price value, returns False 
        if it is near some previously discovered key level
        '''
        
        # levels = support + resistance
        return np.sum([abs(l - x[1]) < s for x in levels]) == 0

    for i in range(2, df.shape[0] - 2):
        # sprev = start if not support else df.iloc[support[-1][0]].name
        # rprev = start if not resistance else df.iloc[resistance[-1][0]].name
        prev = start if not levels else df.iloc[levels[-1][0]].name
        ix = df.iloc[i].name
        high = df.loc[ix, 'High']
        low = df.loc[ix, 'Low']
        d = pd.Timedelta(ix - prev).days # days it takes level to form

        if isSupport(df, i):
            l = df['Low'][i]
            if isFarFromLevel(l):
                # d = pd.Timedelta(ix - sprev).days # days it takes level to form
                support.append((i, l, d))
                levels.append((i, l, d))
                n_spt = 0
                df.loc[ix, 'Support'] = l
                df.loc[ix, 'S-R Formation (Days)'] = d
                df.loc[ix, 'S-R Volume'] = df.loc[prev: ix, 'Volume'].sum()
                df.loc[ix, 'S-R Tests'] = n_spt
                # print(f'Support: {l:,.2f} formed on {ix.date()} took {d} days to form')
        elif isResistance(df, i):
            l = df['High'][i]
            if isFarFromLevel(l):
                # d = pd.Timedelta(ix - rprev).days # days it takes level to form
                resistance.append((i, l, d))
                levels.append((i, l, d))
                n_rst = 0
                df.loc[ix, 'Resistance'] = l
                df.loc[ix, 'S-R Formation (Days)'] = d
                df.loc[ix, 'S-R Volume'] = df.loc[prev: ix, 'Volume'].sum()
                df.loc[ix, 'S-R Tests'] = n_rst
                # print(f'Resistance: {l:,.2f} formed on {ix.date()} took {d} days to form')
        
        if support and resistance:
            s_ix, spt, sd = support[-1]
            r_ix, rst, rd = resistance[-1]
            sdt = df.iloc[s_ix].name
            rdt = df.iloc[r_ix].name
            
            # Support tested
            if high >= spt and low <= spt:
                n_spt += 1
                row = df.iloc[s_ix].name
                df.loc[row, 'S-R Tests'] = n_spt
                print(f'{ix.date()}, Support: {spt:,.2f}, Tests: {n_spt}, Formed: {sdt.date()}, Delta: {sd}')
            
            # Resistance tested
            if high >= rst and low <= rst:
                n_rst += 1
                row = df.iloc[r_ix].name
                df.loc[row, 'S-R Tests'] = n_rst
                print(f'{ix.date()}, Resistance: {rst:,.2f}, Tests: {n_rst}, Formed: {rdt.date()}, Delta: {rd}')
            
    cols = ['Support', 'Resistance', 'S-R Formation (Days)', 'S-R Volume', 'S-R Tests']
    cols = df.columns[:-5].to_list() + cols
    df = df[cols]
                  
    return support, resistance, df


In [8]:
def sr_analysis(df, support, resistance):
    pd.options.display.float_format = "{:,.2f}".format
    df.drop(columns=['Adj Close'], inplace=True)
    levels = support + resistance
    levels.sort(key=itemgetter(0))
    levels_d = {'Support': {k:v for k,v in support},
                'Resistance': {k:v for k,v in resistance}}
    s_ix = 0
    r_ix = 0
    
    # for lvl, ld in levels_d.items():
    #     prev = df.iloc[0].name
    #     for i, l in ld.items():
    #         ix = df.iloc[i].name
    #         # df.loc[ix: , lvl] = l
    #         tdelta = pd.Timedelta(ix - prev)
    #         print(f'{lvl}, ix: {ix}, timedelta: {tdelta}')
    #         prev = ix
    #     # print(i, l)
    #     ix = df.iloc[i].name
    #     print(f'{ix.date()}, {l:,.2f}')

    for i in range(2, df.shape[0] - 2):
        ix = df.iloc[i].name
        high = df.loc[ix, 'High']
        low = df.loc[ix, 'Low']
        spt = df.loc[ix, 'Support']
        rst = df.loc[ix, 'Resistance']
        
        if i == support[s_ix][0]:
            s_ix = s_ix + 1 if s_ix < len(support) - 1 else s_ix
            print(f's_ix: {s_ix}, {len(support)}')
            nxt_s = df.iloc[support[s_ix][0]].name
        if i == resistance[r_ix][0]:
            # print(resistance[r_ix][0])
            r_ix = r_ix + 1 if r_ix < len(resistance) - 1 else r_ix
            nxt_r = df.iloc[resistance[r_ix][0]].name

        if low > rst:
            df.loc[ix: nxt_r, 'Support'] = rst
            # print(f'R-S {i}\n', df.loc[ix, ['High', 'Low', 'Support', 'Resistance']])
        if high < spt:
            df.loc[ix: nxt_s, 'Resistance'] = spt
            # print(f'S-R {i}\n', df.loc[ix, ['High', 'Low', 'Support', 'Resistance']])

    return df.iloc[levels[0][0]:]



    
    

In [12]:
ta_df, support, resistance = sr_levels(df, start, end)
ta_df = sr_analysis(ta_df, support, resistance)
ta_df.tail(50)

s_ix: 1, 2
s_ix: 1, 2


,Open,High,Low,Close,Volume,Support,Resistance
Date,,,,,,,
2022-11-30,151.38,154.98,149.10,154.98,2867700,149.74,149.74
2022-12-01,156.73,156.99,154.42,156.12,1372400,149.74,156.99
2022-12-02,153.97,156.73,153.59,155.67,1183200,124.82,156.99
2022-12-05,153.84,153.89,151.25,152.70,1124700,124.82,156.99
2022-12-06,152.15,153.00,149.91,151.35,1172300,124.82,156.99
2022-12-07,151.23,154.35,151.13,153.73,1011100,124.82,156.99
2022-12-08,154.42,156.99,153.48,156.28,1289900,124.82,156.99
2022-12-09,155.67,156.70,152.68,152.95,1020200,124.82,156.99
2022-12-12,154.01,155.98,153.43,155.33,1071400,124.82,156.99


In [173]:
def calculate_trends(ticker, start, end, period, short_ma, inter_ma, long_ma):
    if period.endswith('min'):
        df = get_intraday_ticker_data(ticker)
    else:
        df = get_ticker_data(ticker)[start : end]

    if period != 'Daily':
        if period == 'Weekly':
            fmt = 'W-MON'
        elif period == 'Monthly':
            fmt = 'BMS'
        else:
            fmt = period.split('min')[0] + 'T'

        ix = df.asfreq(fmt).index
        open_ = df['Open'].resample(fmt).first()
        close = df['Close'].resample(fmt).last()
        high = df['High'].resample(fmt).max()
        low = df['Low'].resample(fmt).min()
        volume = df['Volume'].resample(fmt).sum()
        d = dict(Open=open_, High=high, Low=low, Close=close, Volume=volume)
        df = pd.DataFrame(d, index=ix)

    df[f'MA{short_ma}'] = df['Close'].rolling(short_ma).mean()
    df[f'MA{inter_ma}'] = df['Close'].rolling(inter_ma).mean()
    df[f'MA{long_ma}'] = df['Close'].rolling(long_ma).mean()

    return df

In [79]:
def calculate_trends(ticker, start, end, period, short_ma, inter_ma, long_ma):
    if 'Min' in period:
        df = get_intraday_ticker_data(ticker, period)[start : end]
    else:
        df = get_ticker_data(ticker)[start : end]
        df.drop(columns=['Ticker', 'Adjclose'], inplace=True)

    MAs = [short_ma, inter_ma, long_ma]

    for ma in MAs:
        df[f'MA{ma}'] = df['Close'].rolling(ma).mean()

    golden, death = [], []

    df['signal'] = 0
    df['signal'][long_ma:] = np.where(df[f'MA{inter_ma}'][long_ma:] > df[f'MA{long_ma}'][long_ma:], 1, 0)
    df['crossover'] = df['signal'].diff()  
    
    if df['crossover'].any():
        cross = df['crossover'].to_list()
        try:
            val = cross.index(1)
            golden.append(ticker)
            # pprint(cross)
        except ValueError:
            death.append(ticker)

        print('golden cross:')
        pprint(golden)
        print('death cross:')
        pprint(death)

    return df

In [84]:
ticker = 'abbv'
df = get_ticker_data(ticker)
end = dt.now()
years = 0.5
start = end - timedelta(365 * years)
time_frame, short_ma, inter_ma, primary_ma = 'daily', 10, 20, 50

In [85]:
ta_df = calculate_trends(ticker, start, end, time_frame, short_ma, inter_ma, primary_ma)
# ta_df.head(60)
ta_df[ta_df['crossover'] == 1]

golden cross:
['abbv']
death cross:
[]


,Open,High,Low,Close,Volume,MA10,MA20,MA50,signal,crossover
2022-10-24,148.710007,151.529999,148.699997,150.889999,5139700,144.037,142.231001,140.891801,1,1.0


In [11]:
support, resistance, ta_df = sr_levels(df, start, end)

2022-02-17, Resistance: 160.35, Tests: 1, Formed: 2022-02-17, Delta: 3
2022-02-18, Support: 151.65, Tests: 1, Formed: 2022-02-14, Delta: 9
2022-02-22, Support: 151.65, Tests: 2, Formed: 2022-02-14, Delta: 9
2022-02-23, Support: 151.65, Tests: 3, Formed: 2022-02-14, Delta: 9
2022-02-24, Support: 139.50, Tests: 1, Formed: 2022-02-24, Delta: 7
2022-03-07, Support: 139.50, Tests: 2, Formed: 2022-02-24, Delta: 7
2022-03-08, Support: 133.57, Tests: 1, Formed: 2022-03-08, Delta: 12
2022-03-18, Resistance: 160.35, Tests: 2, Formed: 2022-02-17, Delta: 3
2022-03-21, Resistance: 160.35, Tests: 3, Formed: 2022-02-17, Delta: 3
2022-03-24, Resistance: 160.35, Tests: 4, Formed: 2022-02-17, Delta: 3
2022-03-29, Resistance: 170.83, Tests: 1, Formed: 2022-03-29, Delta: 21
2022-05-12, Support: 102.41, Tests: 1, Formed: 2022-05-12, Delta: 44
2022-05-17, Resistance: 115.80, Tests: 1, Formed: 2022-05-17, Delta: 5
2022-05-24, Support: 102.41, Tests: 2, Formed: 2022-05-12, Delta: 44
2022-05-31, Resistance: 11

In [18]:
ta_df.tail(50)

,Open,High,Low,Close,Volume,Support,Resistance,S-R Formation (Days),S-R Volume,S-R Tests
2022-11-17,95.370003,96.970001,94.029999,94.849998,82617900,NaN,NaN,NaN,NaN,NaN
2022-11-18,95.949997,95.989998,92.480003,94.139999,72428200,NaN,NaN,NaN,NaN,NaN
2022-11-21,93.970001,95.019997,90.589996,92.459999,84330300,90.589996,NaN,12.0,9.140062e+08,5.0
2022-11-22,92.620003,93.349998,90.870003,93.199997,62192000,NaN,NaN,NaN,NaN,NaN
2022-11-23,93.239998,94.580002,92.830002,94.129997,59414700,NaN,NaN,NaN,NaN,NaN
2022-11-25,93.790001,94.430000,93.070000,93.410004,35088600,NaN,NaN,NaN,NaN,NaN
2022-11-28,93.930000,96.400002,93.430000,93.949997,74943100,NaN,NaN,NaN,NaN,NaN
2022-11-29,94.040001,94.410004,91.440002,92.419998,65567300,NaN,NaN,NaN,NaN,NaN
2022-11-30,92.470001,96.540001,91.529999,96.540001,102805800,NaN,NaN,NaN,NaN,NaN
2022-12-01,96.989998,97.230003,94.919998,95.500000,68488000,NaN,97.230003,10.0,5.528298e+08,9.0


In [119]:
ta_df.loc[start, 'Resistance'].notna()

KeyError: datetime.datetime(2022, 1, 31, 17, 48, 31, 465269)

In [166]:
def calculate_trends(ticker, start, end, time_frame, short_ma, inter_ma, primary_ma):
    df = get_ticker_data(ticker)[start : end]
    df.drop(columns=['ticker', 'adjclose'], axis=1, inplace=True)
    df.columns = df.columns.str.title()  

    if time_frame != 'Daily':
        if time_frame == 'Weekly':
            fmt = 'W-MON'
        elif time_frame == 'Monthly':
            fmt = 'BMS'

        ix = df.asfreq(fmt).index
        open_ = df['Open'].resample(fmt).first()
        close = df['Close'].resample(fmt).last()
        high = df['High'].resample(fmt).max()
        low = df['Low'].resample(fmt).min()
        volume = df['Volume'].resample(fmt).sum()
        d = dict(Open=open_, High=high, Low=low, Close=close, Volume=volume)
        df = pd.DataFrame(d, index=ix)

    support, resistance, df = sr_levels(df, start, end)
    levels = support + resistance

    df[f'MA{short_ma}'] = df['Close'].rolling(short_ma).mean()
    df[f'MA{inter_ma}'] = df['Close'].rolling(inter_ma).mean()
    df[f'MA{primary_ma}'] = df['Close'].rolling(primary_ma).mean()

    # for s,r in itertools.zip_longest(support, resistance):
    #     print(s,r)

    # for row in df.index:
    #     # row = df.iloc[i].name
    #     low = df.loc[row, 'Low']
    #     high = df.loc[row, 'High']
    #     support = df.loc[row, 'Support']
    #     resistance = df.loc[row, 'Resistance']
        # print(row, low, high, support, resistance)
        # if low > resistance:
        #     print(f'R - S: {row.date()}')
        #     # print(f'resistance: {resistance}')
        #     support = resistance
        # if high < support:
        #     print(f'S - R: {row.date()}')
        #     resistance = support

    return df, levels

In [183]:
ta_df.loc['2021-04-14', ['High', 'Low', 'Support', 'Resistance']]

High          260.263336
Low           242.676666
Support       240.369995
Resistance    240.369995
Name: 2021-04-14 00:00:00, dtype: float64

In [175]:
# # pd.set_option('display.max_rows', None)
# support, resistance, ta_df = sr_levels(df, start, end)
# ta_df.iloc[short_ma:]

C:\Users\pnjor\AppData\Local\Temp\ipykernel_9236\4069413257.py:1: FutureWarning:

this method is deprecated in favour of `Styler.format(precision=..)`



,open,high,low,close,adjclose,volume,ticker
2010-06-29 00:00:00,1.27,1.67,1.17,1.59,1.59,281494500,TSLA
2010-06-30 00:00:00,1.72,2.03,1.55,1.59,1.59,257806500,TSLA
2010-07-01 00:00:00,1.67,1.73,1.35,1.46,1.46,123282000,TSLA
2010-07-02 00:00:00,1.53,1.54,1.25,1.28,1.28,77097000,TSLA
2010-07-06 00:00:00,1.33,1.33,1.06,1.07,1.07,103003500,TSLA
2010-07-07 00:00:00,1.09,1.11,1.00,1.05,1.05,103825500,TSLA
2010-07-08 00:00:00,1.08,1.17,1.04,1.16,1.16,115671000,TSLA
2010-07-09 00:00:00,1.17,1.19,1.10,1.16,1.16,60759000,TSLA
2010-07-12 00:00:00,1.20,1.20,1.13,1.14,1.14,33037500,TSLA
2010-07-13 00:00:00,1.16,1.24,1.13,1.21,1.21,40201500,TSLA


In [4]:
yf.download('a', interval='1wk')

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1999-11-15 00:00:00-05:00,32.546494,35.765381,28.478184,28.880545,24.674318,77780526
1999-11-22 00:00:00-05:00,29.551144,31.473534,28.612303,29.461731,25.170860,19126178
1999-11-29 00:00:00-05:00,29.327612,32.680614,29.014664,31.831186,27.195219,17453611
1999-12-06 00:00:00-05:00,32.367668,33.217094,31.697067,32.010014,27.348013,10048404
1999-12-13 00:00:00-05:00,32.546494,33.798283,29.327612,32.859444,28.073721,15559180
...,...,...,...,...,...,...
2023-01-09 00:00:00-05:00,149.690002,158.600006,147.199997,156.919998,156.919998,5808000
2023-01-16 00:00:00-05:00,156.679993,158.160004,152.139999,155.919998,155.919998,5114000
2023-01-23 00:00:00-05:00,156.000000,159.589996,152.830002,155.690002,155.690002,4255100


In [43]:
unaware = dt(2011, 8, 15, 8, 15, 12, 0)
aware = dt(2011, 8, 15, 8, 15, 12, 0, pytz.UTC)
now_aware = unaware.replace(tzinfo=timezone('EST'))
now_aware

datetime.datetime(2011, 8, 15, 8, 15, 12, tzinfo=<StaticTzInfo 'EST'>)